Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [16]:
from random import choices
from random import random, choice, randint
import lab9_lib
from copy import copy
import tqdm
import numpy as np

In [17]:
#Problem
PROBLEM = 5
FITNESS_FUNCTION = lab9_lib.make_problem(PROBLEM)

class Individual():
    def __init__(self, genotype: list[bool]) -> None:
        self.genotype = genotype
        self.fitness = FITNESS_FUNCTION(self.genotype)
        
    def __str__(self) -> str:
        return f"{''.join(str(g) for g in self.genotype)}: {self.fitness:.2%}"#Evolutionary Algorithm

# Genetic Algorithm

In [18]:
class EA():
    def __init__(self, tournament_size, mutation_probability, genome_size, population_size,offspring_size, crossover="one_cut_xover", mutation_strength=1, selection_type="tournament", type="+") -> None:
        self.tournament_size = tournament_size
        self.mutation_probability = mutation_probability
        self.genome_size = genome_size
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.mutation_strength = mutation_strength
        self.selection_type = selection_type
        self.type=type
        # select crossover function
        if crossover == "one_cut_xover":
            self.crossover = self.one_cut_xover
        elif crossover == "n_cut_xover":
            self.crossover = self.n_cut_xover
        elif crossover == "uniform_xover":
            self.crossover = self.uniform_xover
        else:
            raise ValueError("Unknown crossover function")
    
    def select_parent(self, population: list[Individual]) -> Individual:
        if self.selection_type == "tournament":
            return self.tournament(population)
        elif self.selection_type == "roulette":
            return self.roulette(population)
        
    def tournament(self, population: list[Individual]) -> Individual:
        pool = [choice(population) for _ in range(self.tournament_size)]
        champion = max(pool, key=lambda i: i.fitness)
        return champion
    
    def roulette(self, population: list[Individual]) -> Individual:
        fitnesses = [i.fitness for i in population]
        total_fitness = sum(fitnesses)
        probabilities = [f/total_fitness for f in fitnesses]
        return choices(population, weights=probabilities)[0]

    def mutate(self, ind: Individual) -> Individual:
        offspring_genotype = copy(ind.genotype)
        for i in range(self.mutation_strength):
            pos = randint(0, self.genome_size-1)
            offspring_genotype[pos] = 1 - offspring_genotype[pos]
        return Individual(offspring_genotype)

    def one_cut_xover(self, ind1: Individual, ind2: Individual) -> Individual:
        cut_point = randint(0, self.genome_size-1)
        offspring = Individual(genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
        assert len(offspring.genotype) == self.genome_size
        return offspring
    
    def n_cut_xover(self, ind1: Individual, ind2: Individual, n=5) -> Individual:
        cut_points = sorted([randint(0, self.genome_size-1) for _ in range(n)])
        o_genotype = []
        pointer=0
        for point in cut_points:
            o_genotype += ind1.genotype[pointer:point]
            pointer=point
            ind1, ind2 = ind2, ind1
        offspring = Individual(genotype=o_genotype + ind1.genotype[pointer:])
        assert len(offspring.genotype) == self.genome_size
        return offspring
    
    def uniform_xover(self, ind1: Individual, ind2: Individual) -> Individual:
        offspring = Individual(genotype=[choice([ind1.genotype[i], ind2.genotype[i]]) for i in range(self.genome_size)])
        assert len(offspring.genotype) == self.genome_size
        return offspring

    def population(self):
        return [Individual(choices([0, 1], k=self.genome_size)) for _ in range(self.population_size)]

    def offsprings(self, population: list[Individual], parents=2) -> list[Individual]:
        offspring = []
            
        while len(offspring) < self.offspring_size:
            # Mutation
            if random() < self.mutation_probability:
                parent=self.select_parent(population)
                #print(f"p: {parent}")
                o=self.mutate(parent)
                #print(f"o: {o}")
                
            # Crossover
            else:
                parent1 = self.select_parent(population)
                #print(f"p1: {parent1}")
                parent2 = self.select_parent(population)
                #print(f"p2: {parent2}")
                o=self.crossover(parent1, parent2)
                #print(f"of: {o}")
            offspring.append(o)
        return offspring


    def generation(self, population: list[Individual], elitism_factor=1) -> list[Individual]:
        offspring = self.offsprings(population)
        if self.type=="+":
            pool = sorted(population + offspring, key=lambda i: i.fitness, reverse=True)
        elif self.type==",":
            pool = sorted(offspring, key=lambda i: i.fitness, reverse=True)
        return pool[:int(self.population_size*elitism_factor)]+choices(pool[int(self.population_size*elitism_factor):], k=int(self.population_size*(1-elitism_factor)))
    
    def run(self, n_generations: int, adaptive_mutation=False, elitism_factor=1) -> Individual:
        population = self.population()
        pbar = tqdm.trange(n_generations, unit="Generations")
        best_fitness=0
        for i in pbar:
            population = self.generation(population, elitism_factor=elitism_factor)
            prev_fitness=best_fitness
            best_fitness=population[0].fitness
            if adaptive_mutation:
                if best_fitness-prev_fitness==0 and self.mutation_probability<0.8:
                    self.mutation_probability=self.mutation_probability*1.1
                elif best_fitness-prev_fitness>0 and self.mutation_probability>0.01:
                    if self.mutation_probability>0.8:
                        self.mutation_probability=0.1
                    self.mutation_probability=self.mutation_probability*0.9
            #print([str(j.fitness) for j in population])
            #pbar.set_postfix({"best_fitness": best_fitness, "fitness_calls":FITNESS_FUNCTION.calls, "best_individual": str(population[0])})
            pbar.set_postfix({"best_fitness": best_fitness, "fitness_calls":FITNESS_FUNCTION.calls, "mutation_rate":self.mutation_probability})
            if best_fitness== 1.0:
                break
        pbar.close()
        return population[0]

In [19]:
  
POPULATION_SIZE = 20
OFFSPRING_SIZE = 10
TOURNAMENT_SIZE = POPULATION_SIZE//5
MUTATION_PROBABILITY = .1
GENOME_SIZE = 1000
MUTATION_STRENGTH=1
GENERATIONS=100000
ADAPTIVE_MUTATION=True
SELECTION_TYPE="tournament"
ELITISM_FACTOR=0.8
TYPE="+"
#CROSSOVER="one_cut_xover"
#CROSSOVER="n_cut_xover"
CROSSOVER="uniform_xover"

In [20]:
FITNESS_FUNCTION._calls=0
evolutionary_algorithm = EA(TOURNAMENT_SIZE, MUTATION_PROBABILITY, GENOME_SIZE, POPULATION_SIZE, OFFSPRING_SIZE, crossover=CROSSOVER, mutation_strength=MUTATION_STRENGTH)
best_individual = evolutionary_algorithm.run(GENERATIONS, adaptive_mutation=ADAPTIVE_MUTATION, elitism_factor=ELITISM_FACTOR)

 28%|██▊       | 28006/100000 [00:38<01:39, 722.41Generations/s, best_fitness=0.309, fitness_calls=280090, mutation_rate=0.806]


KeyboardInterrupt: 